In [6]:
using Plots
using LinearAlgebra
using LaTeXStrings
using HDF5

using Lux, MLUtils, Optimisers, Zygote, OneHotArrays, Random, Statistics, Printf, Reactant
using MLDatasets: MNIST
using SimpleChains: SimpleChains

Reactant.set_default_backend("cpu")

In [2]:
myleakyrelu(x) = leakyrelu(x,0.1)

myleakyrelu (generic function with 1 method)

In [3]:
model = Chain(
    Dense(100,64),
    Dense(64,32),
    Dense(32,60,myleakyrelu)
)

Chain(
    layer_1 = Dense(100 => 64),         # 6_464 parameters
    layer_2 = Dense(64 => 32),          # 2_080 parameters
    layer_3 = Dense(32 => 60, myleakyrelu),  # 1_980 parameters
)         # Total: 10_524 parameters,
          #        plus 0 states.

In [4]:
# Initialize the model parameters and states
rng = Random.default_rng()  # Random number generator
ps, st = Lux.setup(rng, model)

# ps: These are the learnable parameters of your model (like weights and biases in neural networks).
# st: This contains the non-learnable states of your model (like the running mean and variance in BatchNorm).

((layer_1 = (weight = Float32[0.10132206 0.08544423 … 0.0281519 0.0094746305; -0.16883412 -0.11842565 … -0.023698077 -0.13977826; … ; -0.1218989 -0.1476319 … 0.06677648 -0.032968976; -0.0033177682 0.14450046 … -0.15781347 -0.09850688], bias = Float32[-0.0012892603, 0.081913814, -0.040195454, 0.009533584, -0.06345527, 0.008963501, 0.031884205, 0.09437343, -0.080326185, 0.03542757  …  -0.080849946, 0.020556951, 0.09529473, -0.08661993, 0.035990752, 0.02695943, -0.04871583, -0.066234335, 0.05208137, -0.027281772]), layer_2 = (weight = Float32[-0.08565808 0.14090519 … 0.012926232 -0.03458139; -0.06188915 0.21383046 … -0.08361084 0.15074323; … ; -0.20657997 0.09091467 … 0.21633673 -0.12328162; -0.07226994 0.017682238 … -0.16215292 -0.11847095], bias = Float32[-0.075765476, 0.09344193, -0.09282035, 0.012156114, 0.08641355, 0.112708285, 0.0012278706, 0.110777736, -0.114012524, -0.07838319  …  0.10137366, 0.09105086, -0.06663011, 0.06767945, 0.02267243, -0.07684003, 0.022867814, -0.055182233, 

In [5]:
# Let's create a dummy input tensor
batch_size = 10  # Example batch size
dummy_input = rand(Float32, 100, batch_size)  # A random matrix of size (100, batch_size)

# Pass the dummy input through the model
output = model(dummy_input, ps, st)

output[1]# Let's create a dummy input tensor
batch_size = 10  # Example batch size
dummy_input = rand(Float32, 100, batch_size)  # A random matrix of size (100, batch_size)

# Pass the dummy input through the model
output = model(dummy_input, ps, st)

output[1]

60×10 Matrix{Float32}:
  0.216925     0.365129    0.550028    …  -0.0163418   0.225447    0.0876324
 -0.0184894   -0.0644057  -0.0320832      -0.0963397  -0.0264815  -0.0582826
  0.836579     0.185419    0.407654        1.00704     0.775397    0.341812
 -0.00970177   0.360714   -0.0336589       0.24449    -0.0576811   0.507113
  0.0701354    0.992828    0.647236        0.339482    0.829097    0.653223
 -0.0931928   -0.116863   -0.0719385   …  -0.0830875  -0.0688008  -0.0990776
 -0.105169    -0.160983   -0.17239        -0.105989   -0.115978   -0.121647
 -0.127627    -0.0906539  -0.109302       -0.083723   -0.106605   -0.119703
 -0.0335222   -0.0726845  -0.0689177      -0.0541632  -0.0469156  -0.0204456
  1.37643      0.933213    0.758217        0.87068     1.24889     0.870646
  ⋮                                    ⋱                          
  0.0352519    0.268227    0.0444416      -0.0164925  -0.0460499   0.153926
  0.500839     0.618161    0.433297        0.50493    -0.0183806   0.1

## Generando los datos para nuestro sistema 

In [11]:
particles = 3
k=2
list=[0:3 for _ in 1:k]
test_visual_list = []
place= 2
for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end
test_visual_list
 

15-element Vector{Any}:
 [0.0, 1.0, 0.0]
 [0.0, 2.0, 0.0]
 [0.0, 3.0, 0.0]
 [0.0, 0.0, 1.0]
 [0.0, 1.0, 1.0]
 [0.0, 2.0, 1.0]
 [0.0, 3.0, 1.0]
 [0.0, 0.0, 2.0]
 [0.0, 1.0, 2.0]
 [0.0, 2.0, 2.0]
 [0.0, 3.0, 2.0]
 [0.0, 0.0, 3.0]
 [0.0, 1.0, 3.0]
 [0.0, 2.0, 3.0]
 [0.0, 3.0, 3.0]

In [ ]:
# En esta celda tenemos los operadores locales, podemos generar tantos como nos apetezca, pero realmente con generar 
#un sólo par de operadores locales, podemos considearar n pares de valores aleatorios para que el Hamiltoniano sea distintos

particles = 3 # number of qubits
d = 2^particles # dimension of the Hilbert space

# Matrices de Pauli, será nuestra base. 
sigmax = [0 1; 1 0]
sigmay = [0 -im; im 0]
sigmaz = [1 0; 0 -1]
id= [1 0; 0 1] # Matriz identidad

Sigma = Dict(0 => id, 1 => sigmax, 2 => sigmay, 3 => sigmaz) #Diccionario para llamar a las matrices de Pauli

k = 2 # Número de operadores locales que queremos generar

list=[0:3 for _ in 1:k] # Arreglo de tres secuencias de 0 a 3

# Generamos una lista de combinaciones de los operador local A1 
place = 1 # Se utiliza para interactuar únicamente entre el primer y segundo qubit (1er lugar) 
test_visual_list = []

for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end

ArrayA1 = []
for i in 1:length(test_visual_list)
    if test_visual_list[i][place] != test_visual_list[i][place+1] && sum(test_visual_list[i]) != 0
        push!(ArrayA1,test_visual_list[i])
    end
end

# Generamos una lista de combinaciones del operador local A2
place = 2 # Se utiliza para interactuar únicamente entre el segundo y tercero qubit (2do lugar)
test_visual_list = [] # Se tiene que volver a definir esta lista como una lista vacía 

for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end

ArrayA2 = []
for i in 1:length(test_visual_list)
    if test_visual_list[i][place] != test_visual_list[i][place+1] && sum(test_visual_list[i]) != 0
        push!(ArrayA2,test_visual_list[i])
    end
end

# Valores aleatorios 
c = rand(2,1000)*2 .-1 # Estos son los valores de y, los que queremos predecir, cada columna es un valor de y (contiene dos elementos)

xaleatMat1 = rand(1000,length(ArrayA1)).*2 .-1
xaleatMat2 = rand(1000,length(ArrayA2)).*2 .-1


A1P_list = []  # Un arreglo vacío para almacenar matrices
A2P_list = []

for i in 1:1000
    A1P = zeros(2^particles, 2^particles)
    for k in 1:length(ArrayA1)
        A1P += xaleatMat1[i, k] * kron([Sigma[ArrayA1[k][j]] for j in eachindex(ArrayA1[1])]...)
    end
    push!(A1P_list, A1P)  # Guardar la matriz en el arreglo
end

for i in 1:1000
    A2P = zeros(2^particles,2^particles)
    for k in 1:length(ArrayA2)
        A2P += xaleatMat2[i,k]*kron([Sigma[ArrayA2[k][i]] for i in eachindex(ArrayA2[1])]...)
    end
    push!(A2P_list, A2P)  # Guardar la matriz en el arreglo
end

A111 = A1P_list[1] 
A222 = A2P_list[1] # Es por ello que sólo utilizaremos A111 y A222, un sólo par de operadores locales. 

8×8 Matrix{ComplexF64}:
   1.54852+0.0im         -0.253676+0.00109675im  …        0.0+0.0im
 -0.253676-0.00109675im    0.42735+0.0im                  0.0+0.0im
  -1.72699+1.52585im           0.0+0.297693im             0.0+0.0im
       0.0-0.0178144im    0.179005-0.413142im             0.0+0.0im
       0.0+0.0im               0.0+0.0im                  0.0+0.0178144im
       0.0+0.0im               0.0+0.0im         …   0.179005+0.413142im
       0.0+0.0im               0.0+0.0im            -0.489486-1.33645im
       0.0+0.0im               0.0+0.0im             -1.54852+0.0im

In [21]:
# En esta celda lo que haremos es generar un vector de tuplas con la información (x,y) donde x es un vector   
# que contiene los promedios a1 y a2 con el estado 0,1,2,... después el vector y tendrá los valores c1 y c2 
# estos valores se repetirán para todos los estados que consideramos del mismo Hamiltoniano,
# es decir, tomamos para x la mitad de los estados del sistema

n = 1000 # Número de datos que queremos generar 

# Inicializamos un vector vacío para almacenar las 1000 tuplas 
Tup = []

for k in 1 : n

    mat1a1 = zeros(2) # Colocamos un vector de ceros que se va a reiniciar en cada paso 

    # Formamos el Hamiltoniano
    H12 = c[1,k]*A111 + c[2,k]*A222

    # Tomamos los vectores propios 

    vor  = eigen(H12).vectors
    adjvor = adjoint(vor)

    vor[:,1] # Este es el primer vector propio, los vectores propios con eigen se colocan en columnas 
    adjvor[1,:] # Este es el primer vector propio para la adjunta (Recordemos es un vector izquierdo [Fila])

    # Calculamos el valor esperado de la primera mitad de vectores propios

    ExpValA = zeros(ComplexF64, Int((size(vor)[1])/2))
    for i in 1:Int((size(vor)[1])/2)
        ExpValA[i] = transpose(adjvor[i,:])*A111*vor[:,i]
    end

    ExpValB = zeros(ComplexF64,Int((size(vor)[1])/2))  # ExpVal2[1] = a21 = \bra{\phi1_}A_2\ket{\phi_1}
    for i in 1:Int((size(vor)[1])/2)
        ExpValB[i] = transpose(adjvor[i,:])*A222*vor[:,i]
    end

    ValsA = real(ExpValA) # Nos interesa únicamente la parte real, la parte imaginaria suele ser error 
    ValsB = real(ExpValB)
    mat1a1[1] = ValsA[1]
    mat1a1[2] = ValsA[2]

    tk = (mat1a1, c[:,k])
    push!(Tup,tk) # Agregar la tupla al vector 

    ValsCompleto = vcat(ValsA, ValsB) # Por último nuestro vector tendrá la forma de 8x1, para poder agregarlo al batch 
end 

In [29]:
Tup[1][1]

2-element Vector{Float64}:
 -2.6616891144873436
 -2.6445159340119813

In [ ]:
# Extract inputs (x) and outputs (y) from the tuples
x_batch = cat(map(t -> t[1], Tup)...; dims=2)  # Stack inputs along the 4th dimension (batch)
y_batch = cat(map(t -> t[2], Tup)...; dims=2)  # Stack outputs along the 2nd dimension (batch)

# Check the shapes of the resulting batches
println("Shape of x_batch: ", size(x_batch))  # Should be (28, 28, 1, batch_size)
println("Shape of y_batch: ", size(y_batch))  # Should be (10, batch_size)